In [1]:
import pyspark as ps

sc = ps.SparkContext.getOrCreate()

In [6]:
# usuarios = (id_usuario, nickname, pais, fecha_alta) 
# publicaciones = (id_publicacion, id_usuario, id_foto, comentario, fecha)

# Queremos: Encontrar los 10 hashtags que más aparecen publicados únicamente por usuarios Argentinos

usuario1 = (0, 'Tomas', 'Argentina', '-')
usuario2 = (1, 'Matias', 'Brasil', '-')
usuario3 = (2, 'Javier', 'Argentina', '-')
usuario4 = (3, 'Nicolas', 'Jupiter', '-')

publicacion1 = (0, 0, 2, '#Amigos los quiero, #ellos', '-')
publicacion2 = (0, 1, 2, '#Fiesta #Fiesta #Fiesta', '-')
publicacion3 = (0, 0, 2, '#Amigos los quiero #tanto', '-')
publicacion4 = (0, 2, 2, '#Comida #Comida', '-')
publicacion5 = (0, 3, 2, '#Fideos #Fideos', '-')
publicacion6 = (0, 2, 2, '#Amigos #Amigos #Amigos', '-')

usuarios = sc.parallelize([usuario1, usuario2, usuario3, usuario4])
publicaciones = sc.parallelize([publicacion1, publicacion2, publicacion3, publicacion4, publicacion5, publicacion6])

In [7]:
# Función que dado un string devuelve los hashtags como lista

def parseHashtags(str):
    salida = []
    for termino in str.split():
        if(termino[0] == '#'):
            salida.append(termino)

    return salida


usuarios = usuarios.filter(lambda x: x[2] == 'Argentina').map(lambda x : (x[0], x[2]))

publicaciones = publicaciones.map(lambda x: (x[1], x[3]))\
        .flatMap(lambda x: map(lambda y : (x[0],( y, 1)), parseHashtags(x[1])))

usuarios.join(publicaciones).filter(lambda x : x[1][0] == 'Argentina')\
        .map(lambda x : (x[1][1][0], x[1][1][1]))\
        .reduceByKey(lambda x, y : x + y)\
        .takeOrdered(10, lambda x: -x[1])

[('#Amigos', 5), ('#Comida', 2), ('#tanto', 1), ('#ellos', 1)]